# Milk Collection Problem


## Objective and Prerequisites

In this example, you’ll discover how mathematical optimization can be leveraged to solve a capacitated vehicle routing problem: the Milk Collection Problem. With only one tanker truck with limited capacity, you will need to determine the best possible route for the tanker to take to collect milk every day from a set of farms. It’s a complicated problem to solve, but mathematical optimization will help show you the way!

This model is example 23 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 278-281 and 336-337.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

A small milk processing company is committed to collecting milk from 20 farms and taking it back to the depot for processing. The company has one tanker lorry with the capacity to carry 80 000 liters of milk. Eleven of the farms are small and need a collection only every other day. The other nine farms need a collection every day. The positions of the farms in relation to the depot (numbered 1) are given in the following table together with their collection requirements.

![farmCoordinates](https://github.com/Gurobi/modeling-examples/blob/master/milk_collection/farmCoordinates.PNG?raw=1)

The goal is to find the optimal route for the tanker lorry on each day, bearing in mind that it has to:
1. Visit all the ‘every day’ farms.
2. Visit some of the ‘every other day’ farms.
3. Work within its capacity.

On alternate days, it must again visit the ‘every day’ farms and also visit the ‘every other day’ farms not visited on the
previous day.

---
## Model Formulation

### Sets and Indices
$i, j \in \text{Farms} = \{0,1,2, ..,20 \}$: Indices and set of farms. The depot index number is 0.

$\text{everyDay} = \{0,1,2, ..,9 \} \subset \text{Farms}$: Farms that need to visit every day.

$\text{otherDay} = \{10,11, 12, ..,20 \} \subset \text{Farms}$: Farms that need to visit every other day.

$k \in K = \{1,2 \} $: Day type for farms that are visited every other day.

$\text{Edges}= \{(i,j) \in Farms \times Farms \}$: Set of allowed Edges

$S_k \in S  $: Subtour in route of day $k$.

$G = (\text{Farms} , \text{Edges})$: A graph where the set $\text{Farms}$ defines the set of nodes and the set $\text{Edges}$ defines the set of edges.

### Parameters

$d_{i, j} \in \mathbb{R}^+$: Distance from farm $i$ to farm $j$, for all $(i, j) \in \text{Edges}$.

Notice that the distance from farm $i$ to farm $j$ is the same as the distance from farm $j$ to farm $i$, i.e. $d_{i, j} = d_{j, i}$.

$C \in \mathbb{R}^+$: The capacity of the tanker lorry.

$R_i \in \mathbb{R}^+$: Milk collection requirements of farm $i$.

### Decision Variables
$x_{i, j, k} \in \{0, 1\}$: This variable is equal to 1, if the tour on day $k$ connects directly farm $i$ with farm $j$. Otherwise, the decision variable is equal to zero.

$y_{i, k} \in \{0, 1\}$: This variable is equal to 1, if farm $i \in otherDay$ is visited on the tour of day $k \in K$.

### Objective Function
- **Shortest Route**. Minimize the total distance of both routes.

\begin{equation}
\text{Min} \quad Z = \sum_{k \in K} \sum_{(i,j) \in \text{Edges}} \frac{1}{2} d_{i,j} \cdot x_{i,j,k}
\tag{0}
\end{equation}

### Constraints
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the farm $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j, k} = x_{j, i, k} \quad \forall k \in dayType, \; (i, j) \in Edges
\tag{1}
\end{equation}

- **Entering and leaving an every day farm**. For each farm $i$, ensure that this farm is connected to two other farms.

\begin{equation}
\sum_{j: (i,j) \in \text{Edges}} x_{i,j,k} = 2 \quad \forall  i \in everyDay, \; k \in dayType
\tag{2}
\end{equation}

- **Entering and leaving an every other day farm**. For each farm $i$, ensure that this farm is connected to two other farms.

\begin{equation}
\sum_{j: (i,j) \in \text{Edges}} x_{i,j,k}  = 2 \cdot y_{i, k} \quad \forall  i \in otherDay, \; k \in dayType
\tag{3}
\end{equation}

- **Tanker capacity**. The tanker has limited capacity.
\begin{equation}
\sum_{i \in \text{otherDay}} R_{i} \cdot y_{i,k} \leq C -\sum_{i \in everyDay} R_{i} \quad \forall  k \in K
\tag{4}
\end{equation}

- **Farms visited**. Limit on visiting some farms only every other day.

\begin{equation}
y_{i,1} + y_{i,2}  = 1 \quad \forall  i \in \text{otherDay}
\tag{5}
\end{equation}

- **Subtour elimination**. These constraints ensure that for each route there is no cycle.

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \in S
\tag{6}
\end{equation}

Where the subset $S$ is the set of farms visited by the tour, and this tour is defined by the values of the decision variables in the LHS of the inequality.

## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 4.5 MB/s 


In [ ]:
import gurobipy as gp
from gurobipy import GRB

import math
from itertools import combinations

# tested with Python 3.7.0 & Gurobi 9.1.0

### Input data  
We define all the input data for the model.

In [ ]:
# Create a dictionary to capture the farm coordinates (10 miles) and collection requirements (1,000).

farms, coordinates, collect  = gp.multidict({
    0: [(0,0),0],
    1: [(-3,3),5],
    2: [(1,11),4],
    3: [(4,7),3],
    4: [(-5,9),6],
    5: [(-5,-2),7],
    6: [(-4,-7),3],
    7: [(6,0),4],
    8: [(3,-6),6],
    9: [(-1,-3),5],
    10: [(0,-6),4],
    11: [(6,4),7],
    12: [(2,5),3],
    13: [(-2,8),4],
    14: [(6,10),5],
    15: [(1,8),6],
    16: [(-3,1),8],
    17: [(-6,5),5],
    18: [(2,9),7],
    19: [(-6,-5),6],
    20: [(5,-4),6]
})

# List of farm  including the depot
farms = [*range(0,21)]

# List of farms that are visited everyday
everyDay = [*range(0,10)]

# List of farms that are visited each other day
otherDay = [*range(10,21)]

# List of day types
dayType = [1,2]

# Tanker lorry capacity (1,000)
tankerCap = 80

# Every day farms requirements
everyDayReq = 0
for i in everyDay:
    everyDayReq += collect[i]


### Data processing
Here, we calculate the distance for each pair of farms and store it in a Python dictionary.

In [ ]:
# Compute pairwise distance matrix
# numpy linalg norm = euclidean n=2

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(farms, 2)}

### Model Deployment
We now determine the capacitated vehicle routing model for the milk collection problem by defining decision variables, constraints, and objective function.

In [ ]:
# Create the model object m
m = gp.Model('MilkCollection.lp')

# Decision variables:

# Edge variables = 1, if farm 'i' is adjacent to farm 'j' on the tour of day type 'k'.
vars = m.addVars(dist, dayType, vtype=GRB.BINARY, name='x')

# Other day variables = 1, if farm 'i' is visited on the tour of day type 'k'.
other_var = m.addVars(otherDay, dayType, vtype=GRB.BINARY, name='y')

# Symmetry constraints: copy the object (not a constraint)
for i,j,k in vars.keys():
    vars[j, i, k] = vars[i, j, k]

# Every day constraints: two edges incident to an every day farm on tour of day type 'k'.
m.addConstrs((vars.sum(i,'*',k) == 2 for i in everyDay for k in dayType  ), name='everyDay')

# Other day constraints: two edges incident to an other day farm on tour of day type 'k'.
m.addConstrs((vars.sum(i,'*',k) == 2*other_var[i,k] for i in otherDay for k in dayType ), name='otherDay')

# Tanker capacity constraint.
m.addConstrs(( gp.quicksum(collect[i]*other_var[i,k] for i in otherDay ) <= tankerCap-everyDayReq for k in dayType ),
             name='tankerCap')

# Other day farms are visited on day type 1 or 2.
otherDayFarms = m.addConstrs((other_var.sum(i, '*') == 1 for i in otherDay), name='visited')

# Avoid symmetric alternative solutions
other_var[11,1].lb = 1

# Objective function: minimize total distance travel
m.setObjective(gp.quicksum(dist[i,j]*vars[i,j,k] for i,j in dist.keys() for k in dayType), GRB.MINIMIZE)

Restricted license - for non-production use only - expires 2024-10-28


### Finding A Cycle
The following function determines a cycle not connected to the depot.

In [ ]:
# Find the edges from solution values, as a tuplelist for each dayType
def selected(vals):
    s = {k:gp.tuplelist() for k in dayType}
    for i, j, k in vals.keys():
        if vals[i,j,k] > 0.5:
            s[k].append((i,j))
    return s
# Alternately, using comprehension syntax:
#    return {k:gp.tuplelist((i, j) for i, j, k in vals.keys().select('*','*',k) if vals[i,j,k] > 0.5) for k in dayType}

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    nodes = set(i for e in edges for i in e)
    unvisited = list(nodes)
    cycle = list(nodes)
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle): # even if it's the same, we reuse it so that we get the final tour in order
            cycle = thiscycle # New shortest subtour
    return cycle

### Callback Definition
Subtour constraints prevent multiple loops in a tour. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [ ]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # get edges selected in the current solution
        vals = model.cbGetSolution(model._vars)
        edges = selected(vals)
        for k in dayType:
            tour = subtour(edges[k])
            if len(tour) < 0.5*len(edges[k]): # 0.5 due to symmetry: there are both edges i,j and j,i
                # add subtour elimination constr. for farms visited that day
                model.cbLazy(gp.quicksum(model._vars[i, j, k] for i, j in combinations(tour, 2))
                             <= len(tour)-1)

## Solve
We can now optimize the model with the lazy subtour constraints.

In [ ]:
# Optimize the model

m.reset()
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Discarded solution information
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 55 rows, 442 columns and 906 nonzeros
Model fingerprint: 0x97e0e8d8
Variable types: 0 continuous, 442 integer (442 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 12 rows and 32 columns
Presolve time: 0.01s
Presolved: 43 rows, 410 columns, 840 nonzeros
Variable types: 0 continuous, 410 integer (410 binary)

Root relaxation: objective 1.109331e+02, 63 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

## Analysis

We print the optimal total distance traveled and the associated optimal routes for each day type.

In [ ]:
# Print optimal routes and distance traveled

print(f"The optimal distance traveled is: {10*round(m.objVal)} miles.")

vals = m.getAttr('X', vars)
edges = selected(vals)

for k in dayType:
    tour = subtour(edges[k])
    tour.append(0) # return to depot
    print ("Route for day type %i: %s" % (k, " -> ".join(map(str, tour))))

The optimal distance traveled is: 1230 miles.
Route for day type 1: 0 -> 9 -> 5 -> 6 -> 8 -> 7 -> 11 -> 3 -> 15 -> 18 -> 2 -> 4 -> 1 -> 16 -> 0
Route for day type 2: 0 -> 1 -> 17 -> 4 -> 13 -> 2 -> 14 -> 3 -> 12 -> 7 -> 20 -> 8 -> 10 -> 6 -> 19 -> 5 -> 9 -> 0


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC